<a href="https://colab.research.google.com/github/anirbansen3027/SecureAndPrivateAI/blob/master/7_Federated_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install syft

     |████████████████████████████████| 225kB 41.6MB/s 
     |████████████████████████████████| 460kB 53.1MB/s 
     |████████████████████████████████| 2.1MB 46.4MB/s 
     |████████████████████████████████| 81kB 30.0MB/s 
     |████████████████████████████████| 204kB 52.6MB/s 
     |████████████████████████████████| 256kB 50.8MB/s 
     |████████████████████████████████| 389kB 47.6MB/s 
     |████████████████████████████████| 276kB 53.8MB/s 
     |████████████████████████████████| 51kB 20.9MB/s 
     |████████████████████████████████| 122kB 53.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
    

In [0]:
from torchvision import datasets, transforms
import torch as th
from torch import nn,optim
import syft as sy


W0727 18:27:54.302547 140532905056128 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0727 18:27:54.317163 140532905056128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [0]:
alice = sy.VirtualWorker(hook,id = "alice")
bob = sy.VirtualWorker(hook,id = "bob")
charlie = sy.VirtualWorker(hook,id = "charlie")

In [0]:
bob.add_workers([alice,charlie])
alice.add_workers([bob,charlie])
alice.add_workers([bob,alice])

W0727 18:27:55.021756 140532905056128 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0727 18:27:55.023173 140532905056128 base.py:628] Worker charlie already exists. Replacing old worker which could cause                     unexpected behavior
W0727 18:27:55.024314 140532905056128 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0727 18:27:55.025581 140532905056128 base.py:628] Worker charlie already exists. Replacing old worker which could cause                     unexpected behavior
W0727 18:27:55.026689 140532905056128 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0727 18:27:55.027845 140532905056128 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:alice #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [0]:
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)
alice_data = data[2:].send(alice)
alice_target = target[2:].send(alice)

In [0]:
model = nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)
alice_model = model.copy().send(alice)

In [0]:
bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alice_opt = optim.SGD(params=alice_model.parameters(),lr=0.1)

In [0]:
for i in range(10):
  bobs_opt.zero_grad()
  bobs_pred = bobs_model(bobs_data)
  bobs_loss = ((bobs_pred-bobs_target)**2).sum()
  bobs_loss.backward()
  
  bobs_opt.step()
  
  alice_opt.zero_grad()
  alice_pred = alice_model(alice_data)
  alice_loss = ((alice_pred-alice_target)**2).sum()
  alice_loss.backward()
  
  alice_opt.step()

In [0]:
alice_model.move(charlie)
bobs_model.move(charlie)

In [0]:
with th.no_grad():
  model.weight.set_(((alice_model.weight.data+bobs_model.weight.data)/2).get())
  model.bias.set_(((alice_model.bias.data+bobs_model.bias.data)/2).get())

In [0]:
bobs_loss.get(),alice_loss.get()

(tensor(0.0011, requires_grad=True), tensor(0.0040, requires_grad=True))

In [0]:
model.weight.data

tensor([[0.6465, 0.0621]])